In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "123456"
# host = "127.0.0.1"
# port = 27017
# database = "aac"
# collection = "animals"

#db = AnimalShelter(username, password)
db = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
# df.drop(columns=['_id'],inplace=True) Already dropped this column in my read function

# DEBUG
# print(df)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image = "Grazioso Salvare Logo.png"
encoded_image = base64.b64encode(open(image, "rb").read())

app.layout = html.Div([
    html.A(html.Center(html.Img(src="data:image/png;base64,{}".format(encoded_image.decode()), 
                         alt="Grazioso Salvare Logo", height="150px")), href="https:\\www.snhu.edu"),
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Animal Shelter Lookup by Afahri Kerr'))),
    html.Hr(),
    dcc.RadioItems(['Reset', 'Water Rescue', 'Mountain or Wilderness Rescue', 'Disaster Rescue or Individual Tracking'],
                   'Water Rescue', inline=True, id='radio_items'),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True}
                                  for i in df.columns],
                         data=df.to_dict('records'),
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         row_selectable="single",
                         row_deletable=False,
                         selected_rows=[0],
                         page_action="native",
                         page_current=0,
                         page_size=10
                        ),
    html.Br(),
    html.Hr(),
    html.Div(className = 'row', #This creates the pie chart and map and puts them in a row
             style={"display": "flex"},
             children =[
                 html.Div(
                     id='pie-id',
                     className='col s12 m6'
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6'
                 )
             ]
            )
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(Output('datatable-id', "data"),
             [Input('radio_items', 'value')
             ])
def on_click(value): #This will return the correct queries for the different rescue types
    if value == "Water Rescue":
        query = {"breed": {"$in": ["Labrador Retriever Mix",
                                  "Chesapeake Bay Retriever",
                                  "Newfoundland"]
                          },
                "sex_upon_outcome": "Intact Female",
                "$and": [{"age_upon_outcome_in_weeks": {"$gte": 26}},
                        {"age_upon_outcome_in_weeks": {"$lte": 156}}]}
    elif value == "Mountain or Wilderness Rescue":
        query = {"breed": {"$in": ["German Shepherd",
                                  "Alaskan Malamute",
                                  "Old English Sheepdog",
                                  "Siberian Husky",
                                  "Rottweiler"]
                          },
                "sex_upon_outcome": "Intact Male",
                "$and": [{"age_upon_outcome_in_weeks": {"$gte": 26}},
                        {"age_upon_outcome_in_weeks": {"$lte": 156}}]}
    elif value == "Disaster Rescue or Individual Tracking":
        query = {"breed": {"$in": ["Doberman Pinscher",
                                  "German Shepherd",
                                  "Golden Retriever",
                                  "Bloodhound",
                                  "Rottweiler"]
                          },
                "sex_upon_outcome": "Intact Male",
                "$and": [{"age_upon_outcome_in_weeks": {"$gte": 20}},
                        {"age_upon_outcome_in_weeks": {"$lte": 300}}]}
    else:
        query = {}
        
    df = pd.DataFrame.from_records(db.read(query))
    data = df.to_dict('records')
    
    return data

@app.callback(Output('pie-id', 'children'),
              Input('datatable-id', 'data')
              )
def update_graph(viewData): #Creates the pie chart from the selected rescue types
    dff = pd.DataFrame.from_dict(viewData)
    
    return [
        dcc.Graph(
            style={"width": "600px", "height": "600px"},
            figure = px.pie(dff, names="breed")
        )
    ]
                                 

@app.callback(Output('map-id', 'children'),
              [Input('datatable-id', 'data'),
              Input('datatable-id', 'selected_rows')]
              )
def update_map(viewData, index): #Creates the map from the selected rescue types
    dff = pd.DataFrame.from_dict(viewData)
    
    if index is None:
        row = 0
    else: 
        row = index[0]
                                 
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '600px'},
               center=[30.75,-97.48], zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
                   dl.Marker(position=[dff.iloc[row,12],dff.iloc[row,13]],
                             children=[ #Will displey the animals location, breed, and name
                                 dl.Tooltip(dff.iloc[row]["breed"]),
                                 dl.Popup([
                                     html.H1("Animal Name"),
                                     html.P(dff.iloc[row]["name"])
                                 ])
                             ])
               ])
    ]



app.run_server(debug=True)

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/usr/local/anaconda/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/anaconda/lib/python3.9/site-packages/retrying.py", line 49, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
  File "/usr/local/anaconda/lib/python3.9/site-packages/retrying.py", line 212, in call
    raise attempt.get()
  File "/usr/local/anaconda/lib/python3.9/site-packages/retrying.py", line 247, in get
    six.reraise(self.value[0], self.value[1], self.value[2])
  File "/usr/local/anaconda/lib/python3.9/site-packages/six.py", line 719, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.9/site-packages/retrying.py", line 200, in call
    attempt = Attempt(fn(*args, **kwargs), attempt_number, False)
  File "/usr/local/anaconda/lib/python3.9/site-pa